# Dev for using mga1dsm within Saturnian system

In [1]:
import numpy as np
import pykep as pk
from pykep.planet import jpl_lp
from pykep import epoch
import pygmo as pg
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import spiceypy as spice

%matplotlib notebook

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
import pyxplorer as pxp

In [3]:
spice.furnsh(os.path.join(os.environ["SPICE"], "lsk", "naif0012.tls"))
spice.furnsh(os.path.join(os.environ["SPICE"], "spk", "cg_1950_2050_v03.bsp"))

In [4]:
et = spice.utc2et("2044-01-01T00:00:00")
saturn_sv = spice.spkssb(699, et, "ECLIPJ2000")
titan_sv = spice.spkssb(606, et, "ECLIPJ2000")

r_titan_sat = titan_sv[0:3] - saturn_sv[0:3]
v_titan_sat = titan_sv[3:6] - saturn_sv[3:6]

In [5]:
np.linalg.norm(v_titan_sat)

5.5848822304981445

In [6]:
pxp.MU_SATURN

3.79405852e+16

In [7]:
pk.ic2par(r_titan_sat*1e3, v_titan_sat*1e3, pxp.MU_SATURN)

(1221914042.5450587,
 0.028659655999962765,
 0.48280065937311095,
 2.949265038410724,
 3.2731913085559383,
 1.491791176502704)

### Construct Saturn system

In [8]:
saturn_system_list = pxp.saturn_system()
saturn_system_list

{'titan': Planet Name: titan
 Own gravity parameter: 8978138845307.375
 Central body gravity parameter: 37940585200000000
 Planet radius: 2575000
 Planet safe radius: 3575000
 Keplerian planet elements: 
 Semi major axis (AU): 0.0081679908738070735
 Eccentricity: 0.028659655999962765
 Inclination (deg.): 27.662440128212527
 Big Omega (deg.): 168.9804393664231
 Small omega (deg.): 187.54004751915846
 Mean anomaly (deg.): 85.473338328460613
 Elements reference epoch: 2044-Jan-01 00:00:00
 Ephemerides type: Keplerian
 r at ref. = [74005909.532861978, 1079935558.3067718, -563053636.82884848]
 v at ref. = [-5536.0208448477342, 669.44868825434207, 210.2324367740141]}

In [9]:
n_rev = 12
seq = [saturn_system_list["titan"] for el in range(n_rev+1)]

In [10]:
titan_period = saturn_system_list["titan"].compute_period(epoch(2345.3, 'mjd2000')) / 86400  # days
titan_period

15.946856899173898

In [11]:
tof = [
    [0.75*titan_period, 10*titan_period] for el in range(len(seq)-1)
]

t0_earliest = pk.epoch_from_string('2044-01-01 00:00:00.000').mjd2000
t0_latest   = pk.epoch_from_string('2044-01-01 00:00:00.000').mjd2000
    
t0 = [
    t0_earliest,
    t0_latest,
]

In [12]:
tof

[[11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898],
 [11.960142674380425, 159.46856899173898]]

In [13]:
R_incoming, _ = saturn_system_list["titan"].eph(t0_earliest)
vinf_incoming = np.sqrt(2*pxp.MU_SATURN/np.linalg.norm(R_incoming))  # escape velocity w.r.t. Saturn system at Titan orbit

In [14]:
np.linalg.norm(R_incoming), vinf_incoming

(1220150434.449514, 7886.064430152493)

In [15]:
prob_titan = pxp.mga_1dsm_fixed_vexcess(
    seq = seq,
    t0 = t0,                          
    tof = tof,
    vinf = [0.0, 2.0],  # allow for some maneuver at first encounter, km/s
    multi_objective = False, 
    add_vinf_dep = False, 
    add_vinf_arr = True,
    tof_encoding = 'direct',
    vexcess_dep=vinf_incoming,
)

In [16]:
algo = pg.algorithm(pg.sade(
    gen=2000,
))

In [17]:
pop_size = 20

pop = pg.population(prob=prob_titan, size=pop_size)
pop = algo.evolve(pop)

In [18]:
pop.champion_f/1e3

array([11.81424418])

In [19]:
prob_titan.pretty(pop.champion_x)

First Leg: titan to titan
Departure: 2044-Jan-01 00:00:00 (16071.0 mjd2000) 
Duration: 145.7600228315596days
VINF: 1.2383045891705804 km/sec
DSM after 27.565689300371712 days
DSM magnitude: 5179.793073891839m/s

leg no. 2: titan to titan
Duration: 114.61717822965653days
Fly-by epoch: 2044-May-25 18:14:25.972647 (16216.76002283156 mjd2000) 
Fly-by radius: 22.665059616270007 planetary radii
DSM after 85.89021568949923 days
DSM magnitude: 861.5428220844032m/s

leg no. 3: titan to titan
Duration: 149.773249375213days
Fly-by epoch: 2044-Sep-17 09:03:10.171689 (16331.377201061216 mjd2000) 
Fly-by radius: 10.147268468713133 planetary radii
DSM after 34.981801912705365 days
DSM magnitude: 123.21682438823075m/s

leg no. 4: titan to titan
Duration: 55.940104372504116days
Fly-by epoch: 2045-Feb-14 03:36:38.917707 (16481.150450436427 mjd2000) 
Fly-by radius: 5.0645730296390825 planetary radii
DSM after 10.275020322335566 days
DSM magnitude: 357.1906193097308m/s

leg no. 5: titan to titan
Duration:

In [20]:
fig, ax = prob_titan.plot(pop.champion_x, camera=[90,0], get_legend=False)
# set equal axis
xlims = [-0.08, 0.02]
ylims = [-0.02, 0.03]
zlims = [-0.005, 0.015]
pxp.set_equal_axis(ax, xlims, ylims, zlims, scale=1.0)
ax.set(xlabel="x", ylabel="y", zlabel="z")
plt.show()

<IPython.core.display.Javascript object>